# Datalab II <a name='jump'></a>

    Groepsnummer: 2B
    Coach: Marjolijn de Jager
    Namen groepsleden: 
        - Yara Passchier, 22080600
        - Jesse Van Leeuwen, 22096337
        - Alyssa Leliveld, 22111026
        - Bart Jeninga, 22079793

# Table of contents

1. [Sprint 2](#sprint2)
    1. [Beschrijving tabellen](#beschrijftabellen)
        1. [Bepaling gevens van onze club](#gegevensclub)
        2. [Maken functie om gegevens te verzamelen](#functiebepaling)
            1. [Opstellen ranglijst van een seizoen](#ranglijst)
2. [Sprint 3](#sprint3)
    1. [Bepaal met behulp van SQL](#SQL)
        1. [Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen.](#wedstrijdenperseizoen)
        2. [Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen.](#wedstrijden2010)
        3. [Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen.](#puntencompetitie)
        4. [Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst.](#teamranglijst)
    2. [Bepalen invloed teameigenschappen.](#teaminfluence)
        1. [Het mergen van het aantal punten per team en de teameigenschappen.](#dataframeteameigenschappen)
        2. [Bepalen correlatie teameigenschappen en behaalde punten.](#correlatie)

# Sprint 2 <a name="sprint2"></a>

In [3]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')
con = sqlite3.connect("database.sqlite")

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """
    query_output = pd.read_sql_query(query, con)
    return pd.DataFrame(query_output)


In [4]:
# Load the data into a DataFrame
df_country = show_query("SELECT * FROM Country")
df_league = show_query("SELECT * FROM League")
df_match = show_query("SELECT * FROM Match")
df_player = show_query("SELECT * FROM Player")
df_player_attributes = show_query("SELECT * FROM Player_Attributes")
df_team = show_query("SELECT * FROM Team")
df_team_attributes = show_query("SELECT * FROM Team_Attributes")

# Beschrijving tabellen <a name="beschrijftabellen"></a>
Er zijn 7 tabellen:

- country: een tabel met landen. In deze tabel staan alle landen die mee doen met de voetbalwedstrijden.

- league: tabel met de leagues. Deze tabel bevat de data van welke competities er worden gespeeld.

- match: tabel van verschillende wedstrijden. De tabel match bevat informatie over de wedstrijden.

- player: tabel van alle spelers. In deze tabel kan alle informatie over spelers worden gevonden.

- player_attributes: tabel met de eigenschappen van de spelers. De tabel player_attributes bevat informatie over de kwaliteiten en eigenschappen van de spelers.

- team: tabel van de teams. In deze tabel staat alle informatie over de verschillende teams.

- team_attributes: tabel met de eigenschappen van de teams. In deze tabel staat alle informatie van de verschillende teams.

- team_attributes: tabel met de eigenschappen van de teams. En als laatste in deze tabel staan alle kwaliteiten en eigenschappen van de teams.

## 1. Bepaal de volgende gegevens van jouw club: <a name="gegevensclub"></a>
- team naam
- team id
- competitie id
- competitie naam
- De gegevens van alle wedstrijden die jouw club heeft gespeeld.

De eerste stap is het achterhalen van ons team id, SC Heerenveen.

In [5]:
#bepaal team naam, team id, competitie id, competitie naam van SC Heerenveen
query = """

SELECT *

    FROM team

    WHERE team_long_name = 'SC Heerenveen'

"""

show_query(query)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,26562,10228,1913,SC Heerenveen,HEE


Dit team_api_id gebruiken wij vervolgens alle informatie per wedstrijd thuis en uit gespeeld op te halen.

In [6]:
#Bepaal gegevens alle wedstrijden SC Heerenveen
query = """

SELECT *

    FROM match 

    WHERE home_team_api_id = 10228
    OR away_team_api_id = 10228

"""

show_query(query)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,13281,13274,13274,2008/2009,1,2008-08-31 00:00:00,508309,6601,10228,2,...,1.73,3.75,3.40,1.80,4.00,3.4,1.83,3.80,3.40,1.80
1,13291,13274,13274,2008/2009,10,2008-11-09 00:00:00,523739,10217,10228,0,...,1.83,3.75,3.25,1.85,3.75,3.4,1.90,3.80,3.40,1.83
2,13294,13274,13274,2008/2009,11,2008-11-15 00:00:00,523742,10228,9908,3,...,6.00,1.55,3.60,5.00,1.60,3.6,5.00,1.57,3.60,5.50
3,13303,13274,13274,2008/2009,12,2008-11-22 00:00:00,523751,8611,10228,6,...,3.60,2.00,3.40,3.25,2.00,3.3,3.40,2.10,3.25,3.20
4,13314,13274,13274,2008/2009,13,2008-11-29 00:00:00,523762,10228,8640,2,...,2.38,2.50,3.40,2.40,2.60,3.3,2.50,2.60,3.30,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,15682,13274,13274,2015/2016,5,2015-09-13 00:00:00,1983313,8674,10228,3,...,NaN,2.15,3.70,3.20,NaN,NaN,NaN,NaN,NaN,NaN
268,15686,13274,13274,2015/2016,6,2015-09-18 00:00:00,1983317,8464,10228,2,...,NaN,2.40,3.50,2.90,NaN,NaN,NaN,NaN,NaN,NaN
269,15695,13274,13274,2015/2016,7,2015-09-26 00:00:00,1983326,10228,8277,0,...,NaN,2.63,3.70,2.50,NaN,NaN,NaN,NaN,NaN,NaN
270,15704,13274,13274,2015/2016,8,2015-10-02 00:00:00,1983335,9791,10228,2,...,NaN,2.05,3.80,3.50,NaN,NaN,NaN,NaN,NaN,NaN


[Terug naar het begin](#jump)

## 2. Maak een functie waarmee je per seizoen voor een gekozen club o.a. het volgende kunt bepalen: <a name="functiebepaling"></a>

- Aantal wedstrijden in dat seizoen.
- Aantal punten behaald in dat seizoen.
- Gemiddeld aantal punten per wedstrijd.
- Winst-, verlies- en gelijk percentage in uitwedstrijden en thuiswedstrijden.

Hier wordt de functie gemaakt om met 3 parameters direct alle gevraagde informatie terug te krijgen. In de docstring van de functie is te lezen hoe de functie werkt, welke parameters er gebruikt worden en wat er uit de functie komt.

In [7]:
def matchinfo(team_id, season, league_id):
  
    """Bereken de wedstrijdresultaten van een gekozen team voor een gekozen seizoen.
    Parameters:
    -----------
    team_id : int
        Het ID-nummer van het team die je als parameter invult en de resultaten voor worden berekend.
    season : str
        Het seizoen die je als parameter invult waarvan de resultaten worden berekend.
    league_id: int
        Het ID-nummer van de competitie waarin de club speelt en op gefilterd moet worden.
  
    Returns:
    -----------
    pandas.DataFrame
        Een Pandas DataFrame met de wedstrijdresultaten van het team en het seizoen dat je hebt opgegeven bij de parameters. dit zijn de kolommen die het dataframe bevat:
        - totaal_games: het totale aantal van gespeelde wedstrijden in het gekozen seizoen
        - total_points: het totale aantal gehaalde punten in het gekozen seizoen
        - season: seizoen waar de rest van de informatie over gaat
        - team_id: het ID-nummer van het gekozen team
        - average_points: het gemiddelde aantal punten
        - perc_win_home: percentage van de gewonnen thuis wedstrijden
        - perc_draw_home: percentage van gelijk gespeelde thuis wedstrijden
        - perc_loss_home: percentage van de verloren thuis wedstrijden
        - perc_win_away: percentage van de gewonnen uit wedstrijden
        - perc_draw_away: percentage van gelijk gespeelde uit wedstrijden
        - perc_loss_away:  percentage van de verloren thuis wedstrijden
      
    """
    #finding team in home match
    home_match_results = []
    home_team_points = []
    home_df = df_match[(df_match['home_team_api_id'] == team_id) & (df_match['season'] == season) & (df_match['league_id'] == league_id)]
    for index,row in home_df.iterrows():
        if row['home_team_goal'] > row['away_team_goal']:
            home_match_results.append('win')
            home_team_points.append(3)
        elif row['home_team_goal'] == row['away_team_goal']:
            home_match_results.append('draw')
            home_team_points.append(1)
        else:
            home_match_results.append('loss')
            home_team_points.append(0)
 
 
    home_df['match_results'] = home_match_results
    home_df['team_points'] = home_team_points
 
    #finding team in away match
    away_match_results = []
    away_team_points = []
    away_df = df_match[(df_match['away_team_api_id'] == team_id) & (df_match['season'] == season) & (df_match['league_id'] == league_id)]
    for index,row in away_df.iterrows():
        if row['away_team_goal'] > row['home_team_goal']:
            away_match_results.append('win')
            away_team_points.append(3)
        elif row['away_team_goal'] == row['home_team_goal']:
            away_match_results.append('draw')
            away_team_points.append(1)
        else:
            away_match_results.append('loss')
            away_team_points.append(0)
 
    away_df['match_results'] = away_match_results
    away_df['team_points'] = away_team_points
 
    #sum of matches
    home_points = home_df['team_points'].sum()
    away_points = away_df['team_points'].sum()
  
    total_points = home_points + away_points
    total_match = len(home_df) + len(away_df)
    average_points = total_points / total_match
  
    #the percentages for home
    total_match_home = len(home_df)
    if total_match_home > 0:
        perc_win_home = len(home_df[home_df['team_points'] == 3]) / total_match_home * 100
        perc_draw_home = len(home_df[home_df['team_points'] == 1]) / total_match_home * 100
        perc_loss_home = len(home_df[home_df['team_points'] == 0]) / total_match_home * 100
    else:
        perc_win_home = 0
        perc_draw_home = 0
        perc_loss_home = 0
  
    #percentages for away
    total_match_away = len(away_df)
    if total_match_away > 0:
        perc_win_away = len(away_df[away_df['team_points'] == 3]) / total_match_away * 100
        perc_draw_away = len(away_df[away_df['team_points'] == 1]) / total_match_away * 100
        perc_loss_away = len(away_df[away_df['team_points'] == 0]) / total_match_away * 100
    else:
        perc_win_away = 0
        perc_draw_away = 0
        perc_loss_away = 0
  
    #creating the dataframe
    df = pd.DataFrame()
    df['season'] = [season]
    df['team_id'] = team_id
    df['league_id'] = league_id
    df['total_games'] = total_match
    df['total_points'] = total_points
    df['average_points'] = average_points
    df['perc_win_home'] = perc_win_home
    df['perc_draw_home'] = perc_draw_home
    df['perc_loss_home'] = perc_loss_home
    df['perc_win_away'] = perc_win_away
    df['perc_draw_away'] = perc_draw_away
    df['perc_loss_away'] = perc_loss_away
  
    return df

### Pas deze functie toe op een seizoen naar keuze en bepaal de ranglijst van dat seizoen. <a name="ranglijst"></a>

Vervolgens gebruiken wij deze functie om het toe te passen in een loop voor elk team, dit geeft ons een dataframe met de informatie van alle teams en hebben deze gerankschikt.

In [8]:
# Toepassing ranglijst op alle teams voor seizoen 2008/2009

team_list = list(df_match['home_team_api_id'].unique())

df_season = pd.DataFrame()
for team in team_list:
    team_season_results = matchinfo (team, '2008/2009', 13274)
    df_season = pd.concat([df_season,team_season_results])

ranglijst = df_season.sort_values(by = ['total_points'], ascending = False)

In [9]:
ranglijst = ranglijst.dropna().reset_index().set_index('team_id')
del ranglijst['index']
display(ranglijst)

,season,league_id,total_games,total_points,average_points,perc_win_home,perc_draw_home,perc_loss_home,perc_win_away,perc_draw_away,perc_loss_away
team_id,,,,,,,,,,,
10229,2008/2009,13274,34,80.0,2.352941,82.352941,5.882353,11.764706,64.705882,23.529412,11.764706
8611,2008/2009,13274,34,69.0,2.029412,70.588235,23.529412,5.882353,47.058824,29.411765,23.529412
8593,2008/2009,13274,34,68.0,2.000000,76.470588,17.647059,5.882353,47.058824,11.764706,41.176471
8640,2008/2009,13274,34,65.0,1.911765,64.705882,23.529412,11.764706,47.058824,23.529412,29.411765
10228,2008/2009,13274,34,60.0,1.764706,64.705882,29.411765,5.882353,35.294118,23.529412,41.176471
8674,2008/2009,13274,34,56.0,1.647059,64.705882,5.882353,29.411765,35.294118,23.529412,41.176471
9761,2008/2009,13274,34,45.0,1.323529,41.176471,17.647059,41.176471,35.294118,17.647059,47.058824
10235,2008/2009,13274,34,45.0,1.323529,47.058824,29.411765,23.529412,23.529412,23.529412,52.941176
9908,2008/2009,13274,34,44.0,1.294118,47.058824,23.529412,29.411765,17.647059,41.176471,41.176471


[Terug naar het begin](#jump)

# Sprint 3 <a name="sprint3"></a>

## 1. Bepaal met behulp van SQL in deze sprint het volgende:  <a name="SQL"></a>

- Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen.
- Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen.
- Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen.
- Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst.

### Toon het aantal wedstrijden dat jouw team heeft gespeeld per seizoen. <a name="wedstrijdenperseizoen"></a>

In [13]:
#Aantal wedstrijden HC Heerenveen per seizoen.
def wedstrijden(team):
    
    query = f"""
    SELECT season 
         , COUNT(match_api_id) AS aantal_wedstrijden
    FROM Match
    WHERE home_team_api_id = {team}
          OR away_team_api_id = {team}
    GROUP BY season;
    """
    return(show_query(query))

wedstrijden(10228)

,season,aantal_wedstrijden
0,2008/2009,34
1,2009/2010,34
2,2010/2011,34
3,2011/2012,34
4,2012/2013,34
5,2013/2014,34
6,2014/2015,34
7,2015/2016,34


### Toon het aantal wedstrijden van jouw team in het kalenderjaar 2010 per seizoen. <a name="wedstrijden2010"></a>

In [14]:
#Aantal wedstrijden 2010 per seizoen HC Heerenveen
def seizoenwedstrijd(team):
    query = f"""
    SELECT season
         , COUNT(match_api_id) AS aantal_wedstrijden
    FROM Match
    WHERE (home_team_api_id = {team}
          OR away_team_api_id = {team})
          AND date LIKE '2010%'
    GROUP BY season;
    """
    return(show_query(query))

seizoenwedstrijd(10228)

,season,aantal_wedstrijden
0,2009/2010,17
1,2010/2011,19


### wat is deze code hier?

In [19]:
#Punten per seizoen HC Heerenveen
def seizoenspunten(team):
    query1 = f"""
    SELECT season
         , SUM(CASE WHEN home_team_api_id = {team} THEN
                      CASE WHEN home_team_goal > away_team_goal THEN + 3 
                           WHEN home_team_goal = away_team_goal THEN + 1
                           ELSE + 0 END
                    WHEN away_team_api_id = {team} THEN
                      CASE WHEN away_team_goal > home_team_goal THEN + 3
                           WHEN away_team_goal = home_team_goal THEN + 1
                           ELSE + 0 END
                    END) AS aantal_punten
    FROM Match
    WHERE home_team_api_id = {team}
          OR away_team_api_id = {team}
    GROUP BY season;
    """
    return(show_query(query1))
seizoenspunten(10228)

,season,aantal_punten
0,2008/2009,60
1,2009/2010,37
2,2010/2011,41
3,2011/2012,64
4,2012/2013,42
5,2013/2014,57
6,2014/2015,50
7,2015/2016,42


### Toon hoeveel punten ieder team in jouw competitie gehaald heeft per seizoen. <a name="puntencompetitie"></a>

In [20]:
#Punten per seizoen per team
def seizoenspunten(league_id, season):
    query = f"""
        SELECT team_long_name AS team_name
             , SUM(CASE WHEN home_team_api_id = team_api_id THEN
                         CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                        WHEN away_team_api_id = team_api_id THEN
                         CASE WHEN away_team_goal > home_team_goal THEN + 3
                              WHEN away_team_goal = home_team_goal THEN + 1
                              ELSE + 0 END
                        END) AS points
    
    FROM Match
    JOIN Team
        ON home_team_api_id = team_api_id OR away_team_api_id = team_api_id

    WHERE season = "{season}" AND league_id = "{league_id}"
    GROUP BY team_name
    ORDER BY points DESC;


"""
    return(show_query(query))
seizoenspunten(13274, '2008/2009')

,team_name,points
0,AZ,80
1,FC Twente,69
2,Ajax,68
3,PSV,65
4,SC Heerenveen,60
5,FC Groningen,56
6,NAC Breda,45
7,Feyenoord,45
8,FC Utrecht,44
9,Vitesse,43


### Toon op welke plaats jouw team daarmee is geëindigd in de ranglijst. <a name="teamranglijst"></a>

In [24]:
#Plaats in ranglijst HC Heerenveen
def seizoenspunten(league_id, season):
    query = f"""
        SELECT team_long_name AS team_name
                ,ROW_NUMBER() OVER(ORDER BY (SUM(CASE WHEN home_team_api_id = team_api_id THEN
                    CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                    WHEN away_team_api_id = team_api_id THEN
                        CASE WHEN away_team_goal > home_team_goal THEN + 3
                            WHEN away_team_goal = home_team_goal THEN + 1
                            ELSE + 0 END
                    END)))DESC) ranking
                , SUM(CASE WHEN home_team_api_id = team_api_id THEN
                         CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                        WHEN away_team_api_id = team_api_id THEN
                         CASE WHEN away_team_goal > home_team_goal THEN + 3
                              WHEN away_team_goal = home_team_goal THEN + 1
                              ELSE + 0 END
                        END) AS points
    
    FROM Match
    JOIN Team
        ON home_team_api_id = team_api_id OR away_team_api_id = team_api_id

    WHERE season = "{season}" AND league_id = "{league_id}"
    GROUP BY team_name
    ORDER BY points DESC;


"""
    return(show_query(query))
seizoenspunten(13274, '2008/2009')

DatabaseError: Execution failed on sql '
        SELECT team_long_name AS team_name
                ,ROW_NUMBER() OVER(ORDER BY (SUM(CASE WHEN home_team_api_id = team_api_id THEN
                    CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                    WHEN away_team_api_id = team_api_id THEN
                        CASE WHEN away_team_goal > home_team_goal THEN + 3
                            WHEN away_team_goal = home_team_goal THEN + 1
                            ELSE + 0 END
                    END)))DESC) ranking
                , SUM(CASE WHEN home_team_api_id = team_api_id THEN
                         CASE WHEN home_team_goal > away_team_goal THEN + 3
                              WHEN home_team_goal = away_team_goal THEN + 1
                              ELSE + 0 END
                        WHEN away_team_api_id = team_api_id THEN
            

## 2. Bepalen invloed teameigenschappen. <a name="teaminfluence"></a>

### Het mergen van het aantal punten per team en de teameigenschappen.  <a name="dataframeteameigenschappen"></a>

### Bepalen correlatie teameigenschappen en behaalde punten.  <a name="correlatie"></a>

[Terug naar het begin](#jump)